In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import linear_shrinkage
import Util
from Util import evaluate_curret, get_invest_period
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os
import time
import sklearn.covariance
import pickle
from collections import namedtuple, defaultdict
from subprocess import Popen
from multiprocessing import Pool

hi


In [2]:
util_loaded_data = Util.load_data()
ret, ret_nonan, univ, tradeidx, dates = util_loaded_data

In [1]:
assert os.path.exists('./pastRets')
# assert os.path.exists('./outRets')

NameError: name 'os' is not defined

## After this point is the algorithms and experiments, above was just generating data

In [9]:
from estimators import (MTP2_wrapper, MTP2_cov_wrapper,
                        CLIME_wrapper, CLIME_cov_wrapper)

In [15]:
all_methods = {
    # 'MTP2_KT': MTP2_cov_wrapper,
    'CLIME_KT': CLIME_cov_wrapper
}

In [ ]:
# For CLIME KT
# 100 is 8 seconds
# 200 is 88 seconds
# 500 is 88 * (500/200)**2 * (88/(8*4)) = 1512 seconds = 25 minutes...
# 25 * 360 / 7 = 

In [ ]:
# # MTP KT is for N=500 is:
# 1 min (55 seconds) => 6 hrs

### Time estimate

In [16]:
T = 1260
N = 500
with open('pastRets/{}_{}_KTcovs.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)
# This will take around 25 minutes for CLIME_KT
timing_dict = {}
for method_name, m_func in all_methods.items():
    h = 0
    print(method_name, end = ' ')
    args = []
    if method_name == "CLIME_KT":
        args = [T]
    start = time.time()
    cov = m_func(pastRets[h], *args)
    end = time.time()
    print(end-start)
    timing_dict[method_name] = end-start

MTP2_KT 56.22209286689758


In [ ]:
print(timing_dict.items())
print("Total hours: ", sum(timing_dict.values()))

### Run versions

In [17]:
assert os.path.exists("./pickle")
# Otherwise make pickle directory.

In [ ]:
N_T_list = [(500,1260)]

for N, T in N_T_list:
    print(N,T)
    if os.path.isfile('pastRets/{}_{}_KTcovs.pkl'.format(T,N)):
        with open('pastRets/{}_{}_KTcovs.pkl'.format(T,N), 'rb') as f:
            pastCovs = pickle.load(f)
    else:
        assert False
    
run_name = "first_server_run"
KT_or_not = False
for N, T in N_T_list:
    print(N,T)
    with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
        pastRets = pickle.load(f)
        assert len(pastRets) == 360

    def get_covs_all_methods(h):
        print("WORKING ON h={}".format(h))
        for method_name, method in all_methods.items():
            args = []
            if method_name == 'CLIME_KT':
                args = [T]
            cov = method(cov=pastCovs[h], *args)
            fname = 'pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name)
            with open(fname, 'wb') as f:
                pickle.dump(cov, f)
    
    pool = Pool(7)
    pool.map(get_covs_all_methods, range(360))

500 1260
500 1260
WORKING ON h=13
WORKING ON h=26
WORKING ON h=39
WORKING ON h=0
WORKING ON h=52
WORKING ON h=65
WORKING ON h=78
WORKING ON h=14
WORKING ON h=1
WORKING ON h=27
WORKING ON h=79
WORKING ON h=40
WORKING ON h=53
WORKING ON h=66
WORKING ON h=15
WORKING ON h=2


In [ ]:
# def get_portfolio_to_period(end_h, all_methods, T, N, KT_or_not, run_name):
#     with open('outRets/{}_outRets.pkl'.format(N), 'rb') as f:
#         outRets = pickle.load(f)
#     for h in range(end_h):
#         print(h, end= ' ')
#         m = get_momentum_signal(h, N, univ, tradeidx, ret)
#         b = np.mean(m)
#         for method_name in iter_method_names:
#             if h == 0:
#                 print('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name))
#             with open('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name), 'rb') as f:
#                 cov = pickle.load(f)
#             if cov is None or cov == 'FAIL':
#                 curret = None
#             else:
#                 w = optimal_weights_momentum(m, cov, b)
#                 curret = retConstShare(outRets[h], w)
#             method_to_portfolio[method_name].append(curret)
#     return method_to_portfolio

In [ ]:
# method_to_portfolio = get_portfolio_to_period(360, all_methods, 1260, 500, False, "first_server_run")
# pname = 'portfolios_IR/{}_{}_{}_{}_{}'.format(T,N,method_name,KT_or_not,run_name)
# with open(pname.format(1260,500,"MTP2_KT", False, "first_server_run"), "wb") as f:
#     pickle.dump(method_to_portfolio['MTP2_KT'], f)
# with open(pname.format(1260,500,"CLIME_KT", False, "first_server_run"), "wb") as f:
#     pickle.dump(method_to_portfolio['CLIME_KT'], f)